# Image Processing SS 20 - Assignment - 03

### Deadline is 13.5.2020 at 11:55am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import ssl
from urllib.request import urlopen
from skimage.data import astronaut
from skimage.color import rgb2gray, rgb2hsv, hsv2rgb

pylab.rcParams['figure.figsize'] = (12, 12)   # This makes the plot bigger

In [ ]:
img = astronaut() / 255.
img_hsv = rgb2hsv(img)
img_gray = rgb2gray(img)

# Exercise 1 - Implement a Histogram Mapping - 1 Points

In [ ]:
def norm_cdf(arr):
    return arr / arr[-1]

In [ ]:
from math import*

def gamma_mapping(gamma):
    """
    Returns a 1-dimensional numpy array. The value of the array at the n-position 
    is `(n/len(array))**gamma`.
    """
    return norm_cdf(np.linspace(0, 1, 256)**gamma)


def sigmoid_mapping(gain = 10, cutoff = 0.5):
    """
    Returns a 1-dimensional numpy array. The value of the array at the n-position 
    is  `1/(1 + exp^(gain*(cutoff - (n/len(array)))))`.
    """

    return 1 / (1 + np.exp(gain * (cutoff - np.linspace(0, 1, 256))))

In [ ]:
plt.plot(gamma_mapping(1.2))
plt.show()

plt.plot(sigmoid_mapping())
plt.show()

# Exercise 2 - Histogram Transformation - 2 Points

In [ ]:
def apply_pixel_mapping(image, mapping):
    """Returns the image transformed according to the mapping array. 
       `mapping` is a one dimensional numpy array. `image` can be 2 or 3-dimensional.
       The values of the image are in range 0 to 1. 
       If the mapping has for example 255 items, then all pixel with a value from 0 to 1/255 are assigned to 
       the value mapping[0]. If the pixel is between n / 255 and (n+1) / 255 then the value in the output image should 
       be mapping[n]
    """
    
    resImage = np.zeros_like(image)
    
    num_rows = image.shape[0] # number of pixel horizontal
    num_cols = image.shape[1] # number of pixel vertical
    
    if len(image.shape) == 2:
        for row in range(num_rows):
            for col in range(num_cols):
                resImage[row,col] = mapping[floor(image[row,col]*(len(mapping)-1))]
            
    elif len(image.shape) == 3:
        num_channel = image.shape[2]
        for row in range(num_rows):
            for col in range(num_cols):
                for channel in range (num_channel):
                    resImage[row,col,channel] = mapping[floor(image[row,col,channel]*(len(mapping)-1))]
            
    return resImage

In [ ]:
# you can test your `apply_pixel_mapping` function
# The first image should look lighter. The second and third should be the same image.
img_gamma05 = apply_pixel_mapping(img, gamma_mapping(0.5))
plt.subplot(131)
plt.imshow(img_gamma05, cmap='gray')
plt.subplot(132)
plt.imshow(apply_pixel_mapping(img_gamma05, gamma_mapping(2)), cmap='gray')
plt.subplot(133)
plt.imshow(img, cmap='gray')
plt.show()

# Exercise 3 - Implement Histogram Equalisation - 2 Points

Equalise the image given image so that the histogram is mostly unifrom distributed.
You can use `np.histogram` and `np.cumsum`. Checkout the documentation of `np.histogram`, it might have useful optional arguments.

In [ ]:
ssl._create_default_https_context = ssl._create_unverified_context
f = urlopen("https://dl.dropboxusercontent.com/s/ahj4nff6ba8b8sg/lok.jpg?dl=0")
train = rgb2gray(plt.imread(f, format='jpeg'))
plt.imshow(train, cmap='gray')
plt.show()

In [ ]:
hist = np.histogram(train,bins=255,range=(0.0, 1.0), density=True) # get the histogramm of the image

equalisation_mapping = np.zeros(255)  # calculate the right mapping
summ = 0

for i in range(0,255):
    summ += hist[0][i]
    equalisation_mapping[i] = summ

In [ ]:
img_equalised = apply_pixel_mapping(train, equalisation_mapping)

In [ ]:
hist_of_equalised = np.histogram(img_equalised,bins=255,range=(0.0, 1.0), density=True) # get the histogramm of the equalised image


In [ ]:
plt.subplot(131)
plt.plot(np.cumsum(hist[0])) #cumulativ
plt.plot(hist[0])#histogram
plt.subplot(132)
plt.plot(equalisation_mapping)
plt.subplot(133)
plt.plot(np.cumsum(hist_of_equalised[0]))#cumulativ
plt.plot(hist_of_equalised[0])#histogram
plt.show()

In [ ]:
plt.subplot(121)
plt.imshow(img_equalised, cmap='gray')
plt.subplot(122)
plt.imshow(train, cmap='gray')
plt.show()

# Exercise 4 - Implement a hipster filter - 2 Points

1. Convert the image to HSV 
1. Transform the V-Channel with `sigmoid_mapping` and gain = 10.
1. Transform the S-Channel with `sigmoid_mapping` and gain = 10, cufoff=0.35
1. Convert it back to RGB and add the color hsv(0.05, 1, 1) to the image weight by $0.5\cdot(1 - V)$, where V is the resulting V-Channel from step 2.

You can test the code with your own image or the `astronaut()` test image.
If you choose a custom image, you can included it through the `urllib2` library as done with the lok image.
You can use the `rgb2hsv` and `hsv2rgb` functions from the skimage library.

In [ ]:
import copy
from skimage.data import astronaut

img1 = astronaut()   # Get the image

def apply_pixel_mapping_onechannel(image, mapping, channel):
    resImage = copy.deepcopy(image)
    
    num_rows = image.shape[0] # number of pixel horizontal
    num_cols = image.shape[1] # number of pixel vertical
    num_channel = image.shape[2]
    
    for row in range(num_rows):
        for col in range(num_cols):
            resImage[row,col,channel] = mapping[floor(image[row,col,channel]*(len(mapping)-1))]
    return resImage

img2 = rgb2hsv(img1)
img2 = apply_pixel_mapping_onechannel(img2,sigmoid_mapping(20,),2)
plt.imshow(hsv2rgb(img2))     # show the result from step 2
plt.show()

In [ ]:
img3 = apply_pixel_mapping_onechannel(img2,sigmoid_mapping(10,0.35),1)

plt.imshow(hsv2rgb(img3))     # show the result from step 3
plt.show()

In [ ]:
# Convert it back to RGB and add the color hsv(0.05, 1, 1) to the image 
# weight by 0.5⋅(1−V), where V is the resulting V-Channel from step 2.

def apply_pixel_redish(image):
    
    num_rows = image.shape[0] # number of pixel horizontal
    num_cols = image.shape[1] # number of pixel vertical
    
    color = hsv2rgb([[[0.05,1,1]]])
    
    for row in range(num_rows):
        for col in range(num_cols):
            image[row,col,0] += color[0,0,0]*0.5*(1-img2[row,col,2])
            image[row,col,1] += color[0,0,1]*0.5*(1-img2[row,col,2])
            image[row,col,2] += color[0,0,2]*0.5*(1-img2[row,col,2])
    return image

img4 = apply_pixel_redish(hsv2rgb(img3))

plt.imshow((img4))      # show the result from step 4
plt.show()

In [ ]:
# plot the original image
plt.imshow(img1)

# Exercise 5 - Implement your own hipster filter - 3 Points

You have mostly complete artistic freedom what filter you implement. 
The filter should not be trivial. Converting the image only to grayscale is not enough ;) 
You should show off your knowledge of histogram transformations. (Use at least 2 histogram transformations)

In [ ]:
#random landscape picture:
g = urlopen("http://www.pustertal.net/images/fotos/landschaft-pustertal.jpg")
land = plt.imread(g, format='jpeg')
img= land

def minAndsndmin(array):
    minn=array[0]
    sndmin = array[1]
    for i in range(1,len(array)):
        if array[i] < minn:
            sndmin = minn
            minn = array[i]
        elif array[i]< sndmin:
            sndmin = array[i]
    return [minn,sndmin]

#add a vignetting effect to the immage (not perfectly round)
def vignetting(image):
    image = rgb2hsv(image)
    
    num_rows = image.shape[0] 
    num_cols = image.shape[1] 
    
    for row in range(num_rows):
        for col in range(num_cols):
            corners = [row,abs(num_rows-row),col,abs(num_cols-col)]
            distToCorners = minAndsndmin(corners)
            vignettdist = (distToCorners[0]*sqrt(distToCorners[0]**2+distToCorners[1]**2))/100
            image[row,col,2] -= 2000/(vignettdist**2+4000)
            if image[row,col,2] < 0:
                 image[row,col,2] = 0
    
    return hsv2rgb(image)

#increase saturation (higher values exponentially more)
def lomostyle_saturation(image):
    image = rgb2hsv(image)
    array = np.zeros(255)
    #calculate mapping
    for i in range(len(array)):
        intensity = i/200 + exp(i/255)/100
        if intensity > 1:
            intensity = 1
        array[i] = intensity

    image = apply_pixel_mapping_onechannel(image, array, 1)
    
    return hsv2rgb(image)

#increase brightness of upper-mids and heights, decrease lower
def lomostyle_brightness(image):
    image = rgb2hsv(image)
    array = np.zeros(255)
    #calculate mapping
    for i in range(len(array)):
        brightness = (0.0001277/4*i**2)+0.1
        if brightness > 1:
            brightness = 1
        array[i] = brightness

    image = apply_pixel_mapping_onechannel(image, array, 2)
    
    return hsv2rgb(image)
#calculate blue image
def lomostyle_hue(image):
    image = rgb2hsv(image)
    array = np.zeros(255)
    
    for i in range(len(array)):
        hue = 1/2+0.08*i/255
        if hue > 1:
            hue = 1
        array[i] = hue

    image = apply_pixel_mapping_onechannel(image, array, 0)    
    return hsv2rgb(image)
#merge color of blue image with normal image
def add_imagecolors(image1, image2):
    num_rows = image1.shape[0] 
    num_cols = image1.shape[1] 
    num_channels = image1.shape[2]
    hsv_img=rgb2hsv(image1)
    
    for row in range(num_rows):
        for col in range(num_cols):
            for channel in range(num_channels):
                image1[row,col,channel] += 0.8 * image2[row,col,channel] * (1-hsv_img[row,col,2])
                if image1[row,col,channel]>1:
                    image1[row,col,channel] = 1
    return image1
#original
plt.imshow(img)     
plt.show() 
#vignetting
img = vignetting(img)
plt.imshow(img)     
plt.show()
#saturation1
img = lomostyle_saturation(img)
plt.imshow(img)     
plt.show()
#brightness
img = lomostyle_brightness(img)
plt.imshow(img)     
plt.show()
#calculate blue image
img_blue = lomostyle_hue(img)
plt.imshow(img_blue)     
plt.show()
#merge blue with normal
img = add_imagecolors(img,img_blue)
plt.imshow(img)     
plt.show()
#saturation2
img = lomostyle_saturation(img)
plt.imshow(img)     
plt.show()
